In [1]:
import json
import os
import pandas as pd
import numpy as np
from pathlib import Path
import collections
from sklearn.model_selection import train_test_split
from sklearn import metrics

import sys
sys.path.append("../")
from datatools.analyzer import *
from utterance.error_tools import *

from datatools.maneger import DataManager
from datatools.preproc import Preprocessor

In [2]:
# case_set = set("は　が　を　に　で")
case_set = set("が　を　に")
import copy
suit_base =  {
    "が": "[NONE]",
    "を": "[NONE]",
    "に": "[NONE]",
    "V" : ""
}

# def extract_RDF_triple(text, clean_=True):
def predicate_argument_structure_analysis(text, clean_=True):
    if clean_:
        text = clean_text(text)
    doc = nlp(text)
    triple_list = []
    suit_list = []
    for i, token in enumerate( doc ):
        if token.pos_ in ["VERB", "ADJ"]:
            # 受動表現の可能性があるものは回避
            if i<=len(doc)-2 and doc[i+1].pos_ == "AUX" and doc[i+1].lemma_ in ["れる", "られる"]:
                # print(doc)
                continue
            suit = copy.deepcopy(suit_base)
            suit["V"] = token.lemma_
            is_exist_case = False
            for c in token.children:
                if c.dep_ in ["nsubj", "obj", "obl"]:
                    noun = c.lemma_
                    
                    for c2 in c.children:
                        if c2.dep_ == "case" and c2.orth_ in case_set:
                            is_exist_case = True
                        # if c2.dep_ == "case":
                            case = c2.orth_
                            if case == "は":
                                case = "が"
                            triple_list.append( (noun, case, token.lemma_ ))
                            suit[case] = noun
            if is_exist_case:
                # print(suit)
                suit_list.append(suit)
             
    # return triple_list
    return suit_list

def triple_tuple(suit):
    return (suit["が"], suit["を"], suit["に"])

def tuple_add_V2list(p, V):
    suit = list(p)
    suit.append(V)
    return suit

def suit2list(suit):
    return list(suit.values())

In [3]:
from gensim.models import KeyedVectors

w2v_path = "../../corpus/w2v/"
# fasttext
# https://qiita.com/Hironsan/items/513b9f93752ecee9e670
w2v_name =  "dep-ja-300dim"
w2v_name =  "model.vec"
w2v_model = KeyedVectors.load_word2vec_format(w2v_path+w2v_name)

[5242] 2022-01-26 14:51:14,035 Info gensim.models.keyedvectors :loading projection weights from ../../corpus/w2v/model.vec
[5242] 2022-01-26 14:52:17,603 Info gensim.utils :KeyedVectors lifecycle event {'msg': 'loaded (351122, 300) matrix of type float32 from ../../corpus/w2v/model.vec', 'binary': False, 'encoding': 'utf8', 'datetime': '2022-01-26T14:52:17.602822', 'gensim': '4.0.1', 'python': '3.6.9 (default, Jan 26 2021, 15:33:00) \n[GCC 8.4.0]', 'platform': 'Linux-5.4.72-microsoft-standard-WSL2-x86_64-with-Ubuntu-18.04-bionic', 'event': 'load_word2vec_format'}


In [4]:
symbol_path = "../models/utterance/"
symbol_name = "caseframe_symbol2.pickle"
symbolM = DataManager(symbol_path)
SYMBOL_w2v = symbolM.load_data(symbol_name)

success load : ../models/utterance/caseframe_symbol2.pickle


In [5]:
def w2v(word, w2v_model:KeyedVectors, SYMBOL_w2v:dict):
    # 形態素が登録されていたとき
    
    if word in SYMBOL_w2v:
        vector = SYMBOL_w2v[word]
    elif word in w2v_model:
        vector = w2v_model[word]
    else:
        vector = SYMBOL_w2v["[UNK]"]
    return vector
    
def make_X(X_str):
    X = []
    for suit in X_str:
        vector = np.array([w2v(w, w2v_model, SYMBOL_w2v) for w in suit ]).flatten()
        X.append(vector)
    return np.array(X)

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.modules import loss
import torch.optim as optim

In [7]:
class CaseFrameModel(nn.Module):
    def __init__(self, embedding_dim, tagset_size):
        # 親クラスのコンストラクタ。決まり文句
        super(CaseFrameModel, self).__init__()    
        self.embedding_dim = embedding_dim
        self.hid1= embedding_dim*2
        self.hid2 = embedding_dim//2
        # self.fb_dim = 4
        # self.fb_dim = 0
        self.fc1 = nn.Linear(self.embedding_dim, self.hid1)
        self.fc2 = nn.Linear(self.hid1, self.hid2)
        # LSTMの出力を受け取って全結合してsoftmaxに食わせるための１層のネットワーク
        # self.hidden2tag = nn.Linear(self.hid2+self.fb_dim, tagset_size)
        self.hidden2tag = nn.Linear(self.hid2, tagset_size)
    
    def forward(self, x):
        y = F.relu(self.fc1(x))
        y = F.relu(self.fc2(y))
        y = self.hidden2tag( y )
        y = F.log_softmax(y, dim=1)
        return y

In [8]:
model_path = "../models/caseframe/"
model_name = "CF_v2.pickle"
modelM = DataManager(model_path)
model = modelM.load_data(model_name)

success load : ../models/caseframe/CF_v2.pickle


In [9]:
path = "../hand_labeled/"
datalist = ['DCM', 'DIT', 'IRS']
convs = read_conv(path, datalist)

In [10]:
error = "Semantic error"
sys_utt = []
y = []
for conv in convs:
    for ut in conv:
        if ut.is_system() and ut.is_exist_error():
        # if not ut.is_exist_error():
            sys_utt.append(ut.utt)
            if ut.is_error_included(error):
                y.append(1)
            else:
                y.append(0)

In [11]:
len(y)

1349

In [12]:
suit_list = [ predicate_argument_structure_analysis(utt) for utt in tqdm(sys_utt)]

100%|██████████| 1349/1349 [00:19<00:00, 68.08it/s]


In [13]:
y_pred = []
with torch.no_grad():
    for turn in tqdm(suit_list):
        # 述語項の組み合わせがない
        if len(turn)==0:
            y_pred.append(0)
            continue
        x = make_X(turn)
        # 予測
        tens = torch.tensor(x, device='cuda:0').float()
        pred= np.array(model(tens).cpu()).argmax(axis=1)
        if 1 in pred:
            y_pred.append(1)
            print(turn)

    

  4%|▍         | 51/1349 [00:00<00:09, 132.96it/s]

[{'が': '[NONE]', 'を': '[NONE]', 'に': '海', 'V': '行く'}]
[{'が': '実', 'を': '[NONE]', 'に': '[NONE]', 'V': 'なる'}]
[{'が': '[NONE]', 'を': '[NONE]', 'に': '映画', 'V': '教える'}]
[{'が': '[NONE]', 'を': '[NONE]', 'に': '映画', 'V': 'はまる'}]
[{'が': '[NONE]', 'を': 'チケット', 'に': '[NONE]', 'V': '買う'}]
[{'が': '[NONE]', 'を': '[NONE]', 'に': '焼き肉', 'V': '行く'}]
[{'が': '[NONE]', 'を': '私', 'に': '[NONE]', 'V': '連れる'}]
[{'が': '[NONE]', 'を': '活力', 'に': '[NONE]', 'V': '生む'}]
[{'が': 'カロリー', 'を': '[NONE]', 'に': '[NONE]', 'V': '低い'}]
[{'が': 'ラーメン', 'を': '[NONE]', 'に': '[NONE]', 'V': '食べる'}]
[{'が': '麺', 'を': '[NONE]', 'に': '[NONE]', 'V': '美味しい'}]
[{'が': 'パスタ', 'を': '[NONE]', 'に': '[NONE]', 'V': '好き'}]
[{'が': '系', 'を': '[NONE]', 'に': '[NONE]', 'V': '好き'}]
[{'が': '台風', 'を': '[NONE]', 'に': '[NONE]', 'V': '凄い'}]
[{'が': '予防', 'を': '[NONE]', 'に': '[NONE]', 'V': '大切'}]
[{'が': '[NONE]', 'を': '[NONE]', 'に': '海', 'V': '行く'}]
[{'が': '[NONE]', 'を': '[NONE]', 'に': '海', 'V': '行く'}]
[{'が': '[NONE]', 'を': '[NONE]', 'に': '買い物', 'V': '行く'}]
[{

 11%|█         | 150/1349 [00:00<00:03, 301.01it/s]

[{'が': '野菜', 'を': '[NONE]', 'に': '[NONE]', 'V': '高い'}]
[{'が': '冷夏', 'を': '[NONE]', 'に': '[NONE]', 'V': '多い'}]
[{'が': '野菜', 'を': '[NONE]', 'に': '[NONE]', 'V': '高い'}]
[{'が': '質', 'を': '[NONE]', 'に': '[NONE]', 'V': '多い'}]
[{'が': '[NONE]', 'を': '本', 'に': '[NONE]', 'V': '読む'}]
[{'が': '[NONE]', 'を': '[NONE]', 'に': '趣味', 'V': '没頭'}]
[{'が': '[NONE]', 'を': '時間', 'に': '[NONE]', 'V': '費やす'}]
[{'が': 'お金', 'を': '[NONE]', 'に': '[NONE]', 'V': 'ない'}]
[{'が': '一人', 'を': '[NONE]', 'に': '[NONE]', 'V': 'いい'}]
[{'が': '足元', 'を': '[NONE]', 'に': '[NONE]', 'V': '悪い'}]
[{'が': '[NONE]', 'を': '本', 'に': '[NONE]', 'V': '読む'}]
[{'が': '習慣', 'を': '[NONE]', 'に': '[NONE]', 'V': 'ある'}, {'が': '[NONE]', 'を': '小説', 'に': '[NONE]', 'V': '読む'}]
[{'が': '[NONE]', 'を': '名', 'に': '歴史', 'V': '残す'}]
[{'が': '[NONE]', 'を': '猫', 'に': '[NONE]', 'V': '飼う'}]
[{'が': '[NONE]', 'を': '[NONE]', 'に': '人', 'V': '懐く'}]
[{'が': '[NONE]', 'を': '[NONE]', 'に': '年', 'V': '接続'}]
[{'が': '[NONE]', 'を': '四季', 'に': '[NONE]', 'V': '感じる'}]
[{'が': '焦点', 'を': '[

 21%|██▏       | 287/1349 [00:00<00:02, 494.04it/s]

...

[{'が': 'レモン', 'を': '[NONE]', 'に': '[NONE]', 'V': 'なる'}]
[{'が': '私', 'を': '[NONE]', 'に': '[NONE]', 'V': '乗る'}, {'が': '[NONE]', 'を': '広島県', 'に': '[NONE]', 'V': '見る'}]
[{'が': '[NONE]', 'を': '蜜柑', 'に': '前面', 'V': '押し出す'}, {'が': '永谷園', 'を': '[NONE]', 'に': '[NONE]', 'V': '販売'}, {'が': '[NONE]', 'を': '果汁', 'に': '[NONE]', 'V': '使用'}, {'が': '[NONE]', 'を': '違和感', 'に': '商品', 'V': '感ずる'}, {'が': '[NONE]', 'を': '[NONE]', 'に': '気', 'V': 'なる'}]
[{'が': '東北楽天ゴールデンイーグルス', 'を': '[NONE]', 'に': '[NONE]', 'V': '販売'}, {'が': '[NONE]', 'を': 'しらす', 'に': '[NONE]', 'V': '使用'}, {'が': '[NONE]', 'を': '果汁', 'に': '[NONE]', 'V': '使用'}, {'が': '[NONE]', 'を': '鰹', 'に': '[NONE]', 'V': '使用'}, {'が': '[NONE]', 'を': 'ラーメン', 'に': '振り掛け', 'V': 'する'}, {'が': '種類', 'を': '[NONE]', 'に': '[NONE]', 'V': '入る'}]
[{'が': '投手', 'を': '[NONE]', 'に': '[NONE]', 'V': '抜ける'}, {'が': '[NONE]', 'を': '[NONE]', 'に': '東北楽天ゴールデンイーグルス', 'V': '大きい'}]
[{'が': '[NONE]', 'を': '[NONE]', 'に': '投手', 'V': '頑張る'}]
[{'が': '私', 'を': '[NONE]', 'に': '[NONE]', 'V': '勤務'}

 92%|█████████▏| 1239/1349 [00:02<00:00, 1038.14it/s]

[{'が': '[NONE]', 'を': '話', 'に': '[NONE]', 'V': 'する'}]
[{'が': '値段', 'を': '[NONE]', 'に': '[NONE]', 'V': '変わる'}]
[{'が': '[NONE]', 'を': '[NONE]', 'に': '冬場', 'V': 'いい'}]
[{'が': '上', 'を': '[NONE]', 'に': '[NONE]', 'V': '整理'}]
[{'が': '[NONE]', 'を': '[NONE]', 'に': '歌', 'V': 'ある'}]
[{'が': '方', 'を': '[NONE]', 'に': '[NONE]', 'V': '好き'}]
[{'が': '[NONE]', 'を': 'サッカー', 'に': '[NONE]', 'V': 'みる'}]
[{'が': '機会', 'を': '[NONE]', 'に': '[NONE]', 'V': 'ある'}, {'が': '授業', 'を': '[NONE]', 'に': '[NONE]', 'V': '好き'}]
[{'が': '時期', 'を': '[NONE]', 'に': '[NONE]', 'V': '違う'}]
[{'が': '大前', 'を': '[NONE]', 'に': '[NONE]', 'V': 'できる'}]
[{'が': '上', 'を': '[NONE]', 'に': '[NONE]', 'V': '整理'}]
[{'が': '[NONE]', 'を': '何', 'に': '[NONE]', 'V': 'する'}]
[{'が': '[NONE]', 'を': 'morrie', 'に': '[NONE]', 'V': '読む'}]
[{'が': '短編', 'を': '[NONE]', 'に': '[NONE]', 'V': '好き'}]
[{'が': '[NONE]', 'を': 'サーブ', 'に': '[NONE]', 'V': '打つ'}]
[{'が': 'スポーツ', 'を': '[NONE]', 'に': '[NONE]', 'V': '好き'}]
[{'が': '[NONE]', 'を': '欠き氷', 'に': '[NONE]', 'V': '食べる'}]
[{'が

100%|██████████| 1349/1349 [00:02<00:00, 604.23it/s] 

[{'が': '○', 'を': '[NONE]', 'に': '[NONE]', 'V': 'ある'}, {'が': '[NONE]', 'を': '[NONE]', 'に': 'ほり', 'V': '感ずる'}]
[{'が': '[NONE]', 'を': '[NONE]', 'に': '時代', 'V': '戻る'}]
[{'が': 'チャット', 'を': '[NONE]', 'に': '[NONE]', 'V': '好き'}, {'が': '[NONE]', 'を': '[NONE]', 'に': '文', 'V': '書く'}]
[{'が': '誰', 'を': '[NONE]', 'に': '[NONE]', 'V': '歌う'}]
[{'が': '上', 'を': '[NONE]', 'に': '[NONE]', 'V': '整理'}]
[{'が': '電話代', 'を': '[NONE]', 'に': '[NONE]', 'V': 'かかる'}, {'が': 'メール', 'を': '[NONE]', 'に': '[NONE]', 'V': 'いい'}, {'が': '[NONE]', 'を': 'スカイプ', 'に': '[NONE]', 'V': '使う'}]
[{'が': '[NONE]', 'を': '夕食', 'に': '[NONE]', 'V': '食べる'}, {'が': '[NONE]', 'を': '[NONE]', 'に': '一緒', 'V': '行く'}]
[{'が': '[NONE]', 'を': '[NONE]', 'に': 'パー', 'V': 'なる'}]
[{'が': '大前', 'を': '[NONE]', 'に': '[NONE]', 'V': 'できる'}]
[{'が': '大前', 'を': '[NONE]', 'に': '[NONE]', 'V': 'できる'}]
[{'が': '[NONE]', 'を': '[NONE]', 'に': '何処', 'V': 'ある'}, {'が': '持ち', 'を': '[NONE]', 'に': '[NONE]', 'V': '違う'}]


In [14]:
score(y, y_pred)

confusion matrix = 
 [[490 853]
 [  2   4]]
accuracy =  0.36619718309859156
precision =  0.004667444574095682
recall =  0.6666666666666666
f1 score =  0.009269988412514482


- persona のみ

        confusion matrix = 
        [[349 994]
        [  1   5]]
        accuracy =  0.26241660489251295
        precision =  0.005005005005005005
        recall =  0.8333333333333334
        f1 score =  0.009950248756218907

    - 検出

            必要とケースバイケースになってるんですね。ケースバイケースを見ます
            北海道をはじめるかもしれない
            元気は良いんですか？？元気ですかは元気か
            天気が外に出ますよね
            わけは白石が好きですね

In [15]:
for yt, yp, x in zip(y, y_pred, sys_utt):
    if yt==0 and yp:
        print(x)

海に行かないんですね？
スイカは実がなってたことあります
あなたの好きな映画について教えてください
映画にはまるのです
映画館はチケットを買うのです
食事のパートナーはいてるんですね。外食で焼肉に行きますねぇ
あなた、外食のときも私を連れて行ってよ。
夕食は明日の活力を生むから、しっかり食べてね。
野菜はカロリーが低いですよねえ
ラーメンが食べたいです？
麺が美味しいですね
パスタが好きですね
パスタはクリーム系が好きですね
台風が凄いです
手洗いうがいで予防が大切ですよねえ
海に行かないんですね？
海に行ってるんですね。海は一人で行ってます？
家族で買い物に行きます？
家族でどこかに出かけます？
家族で初詣に行ってるんですね。家族で旅行に行ってるのですか？
家族は夕食を食べてるのですか？
ケースバイケースで恋愛スキルを上げたいのです
そういえば、よく旅行に行くと言っていましたが、ケースバイケースで臨機応変に対応するのでしょうか？
お名前はco.jpに添えます
旅行に参加しちゃった
旅行を検討しちゃった
大分別府まで九州横断特急に乗るんでしょうか？
奇遇に驚きますか？
しりとりは一緒に盛り上がりましょうか？
紅玉を求めるかもしれない
そういえば、この間、旅行したと伺いましたが、奇遇に驚きます
旅行が出来るのです
朝から天気が変わりますよねー
付近は紅葉が進むようです
万葉集にも古今和歌集にも沢山恋の歌があるよね。
雨が降った時の臭いはペトリコールという物質が発生するからだよ。
世界で最初にサッカーがワールドカップっていう言葉を使ったんだよ。
飛行機でどこかに行ってるのですか？
一緒がいいですね
味噌汁がご飯にいいですね
歯ごたえが抜群にいいですね
糖度が高いです
ももを探します
曲が好きですね
酵素が強いですね
泳ぐを繰り返すのです
泳ぐを覚えるのです
泳ぐを覚えるんです
泳ぐを繰り返すようですよ
泳ぐを覚えますよねー
熱中症に気をつけないんですか？
冷夏が多いらしいですよ
冷夏は野菜が高いです
冷夏が多いです
冷夏で野菜が高いです
野菜は繊維質が多いですね
読書だそうですが、最近は本を読んでおられますか？
趣味に没頭できるのです
趣味は時間を費やすのです
お金がもったいないです
一人がいいですね
雨で足元が悪いです
読書だそうですが、最近は本を読んでおられますか？